In [29]:
from sklearn import datasets
# coding: utf-8
import datetime
import gc
import math
import os
import seaborn as sns
import time
import warnings
from contextlib import contextmanager

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (auc, average_precision_score, confusion_matrix,
                             mean_squared_error, precision_recall_curve,
                             roc_curve)
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [39]:
iris = datasets.load_iris(as_frame=True)
target_names = iris['target_names']
df = iris['frame']

In [54]:
params ={
    'task': 'train',
    'boosting': 'goss',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.008,
    'subsample': 0.9855232997390695,
    'max_depth': 7,
    'top_rate': 0.9064148448434349,
    'num_leaves': 63,
    'min_child_weight': 41.9612869171337,
    'other_rate': 0.0721768246018207,
    'reg_alpha': 9.677537745007898,
    'colsample_bytree': 0.5665320670155495,
    'min_split_gain': 9.820197773625843,
    'reg_lambda': 8.2532317400459,
    'min_data_in_leaf': 21,
    'verbose': -1,
    'seed':0,
    'bagging_seed':0,
    'drop_seed':0,
    }


In [55]:
folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=42)

In [56]:
train_df = df[df.columns[0:4]]
target = df['target']

In [113]:
# LightGBM GBDT with KFold or Stratified KFold
def oof_lgb(train_df, target, folds, params):
    print(f"Starting LightGBM {train_df.shape}")
    # Cross validation model
    cv_score = 0
    # # Create arrays and dataframes to store results
    # cv_score = 0
    oof_preds = np.zeros(train_df.shape[0])
    # sub_preds = np.zeros(test_df.shape[0])
    # feature_importance_df = pd.DataFrame()
    # feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

    # k-fold
    feature_importance_df = pd.DataFrame()
    for _fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, target)):
        train_x, train_y = train_df.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], target.iloc[valid_idx]
        
            # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)
        

        clf = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        )
        oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        cv_score += mean_squared_error(valid_y, oof_preds[valid_idx]) /folds.n_splits

        # feature importance
        fold_importance_df = pd.DataFrame(data = {'split' : clf.feature_importance(importance_type = 'split'),
                                                'gain' : clf.feature_importance(importance_type = 'gain')}, index= train_df.columns)
        fold_importance_df['fold'] = _fold
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        # return clf, fold_importance_df
    return cv_score / folds.n_splits, feature_importance_df

In [114]:
loss, importance_df = oof_lgb(train_df, target, folds, params)

Starting LightGBM (150, 4)


In [115]:
importance_df

,split,gain,fold
sepal length (cm),14,213.849411,0
sepal width (cm),0,0.000000,0
petal length (cm),36,591.764317,0
petal width (cm),48,816.843140,0
sepal length (cm),14,212.933880,1
sepal width (cm),0,0.000000,1
petal length (cm),40,655.559005,1
petal width (cm),47,814.303799,1
sepal length (cm),12,168.318300,2
sepal width (cm),0,0.000000,2


In [104]:
clf.feature_importance(importance_type = 'split')

array([14,  0, 36, 48], dtype=int32)